Used libraries:

In [167]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import ElasticNet
import optuna
import pickle

## Preprocessing

In [157]:
# Import dataset
train_df = pd.read_csv('../../backend/data/training-housing-dataset.csv')

In [158]:
train_df.head()

In [159]:
# Scaling Data
scaled_columns = ["price", "area", "area_per_room"]
scaler = MinMaxScaler()
train_df[scaled_columns] = scaler.fit_transform(train_df[scaled_columns])

In [160]:
# Creating training and testing datasets
X = train_df.drop(columns="price")
Y = train_df['price']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [161]:
# Creating objective function for model optimization

def objective_elastic(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-4, 1e1)
    l1_ratio = trial.suggest_uniform('l1_ratio', 0.0, 1.0)
    
    # Creating, training and predicting Elastic Net model with suggested hyperparams
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_test)
    mse=mean_squared_error(Y_test, y_pred)
    
    return mse

In [164]:
# Creating study and optimizing
study = optuna.create_study(direction='minimize') # To minimize MSE
study.optimize(objective_elastic, n_trials=100)

In [165]:
print('Best parameters found: ', study.best_params)
print("Best MSE achieved", study.best_value)

In [166]:
# Training final model with best parameters
best_params = study.best_params
final_model = ElasticNet(alpha=best_params['alpha'], l1_ratio=best_params['l1_ratio'], random_state=42)
# Training model on entire set
final_model.fit(X_train, Y_train)

# Model Evaluation
y_pred = final_model.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Test MSE with optimized hyperparameters: {mse}")


In [169]:
r2 = r2_score(Y_test, y_pred)
print(f"Test r2 with optimized hyperparameters: {r2}")

In [168]:
# Saving model for use with website

with open('elastic_net_model.pkl', 'wb') as file:
    pickle.dump(final_model, file)